## Q leaning with value function aproximation

In [23]:
import numpy as np
import gym
import torch
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt
from torch import Tensor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.kernel_approximation import RBFSampler


  Using cached wandb-0.12.13-py2.py3-none-any.whl (1.8 MB)
  Using cached setproctitle-1.2.2.tar.gz (23 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached GitPython-3.1.27-py3-none-any.whl (181 kB)
  Using cached sentry_sdk-1.5.8-py2.py3-none-any.whl (144 kB)
  Using cached protobuf-3.20.0-cp310-cp310-win_amd64.whl (903 kB)
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached promise-2.3-py3-none-any.whl
  Using cached shortuuid-1.0.8-py3-none-any.whl (9.5 kB)
  Using cached PyYAML-6.0-cp310-cp310-win_amd64.whl (151 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached psutil-5.9.0-cp310-cp310-win_amd64.whl (245 kB)
  Using cached gitdb-4.0.9-py3-none-a

  error: subprocess-exited-with-error
  
  × Building wheel for setproctitle (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'setproctitle' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for setproctitle
ERROR: Could not build wheels for setproctitle, which is required to install pyproject.toml-based projects


### The sate of the environment 
- x: the car position
- v: the linear speed of the car
- theta: the angle of the pole
- omega: the angular velocilty of the pole

In [17]:
env = gym.make('CartPole-v1')
env.reset()

# Min and max values for each dimension
print(env.observation_space.high)
print(env.observation_space.low)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


In [22]:
observation_examples = np.array([env.observation_space.sample() for x in range(10000)])

scaler = StandardScaler()

# Feature extraction
feature_generator = FeatureUnion([
    ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
    ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
    ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
    ("rbf4", RBFSampler(gamma=0.5, n_components=100))
])

# feature_generator.fit(scaler.fit_transform(observation_examples))

In [11]:
class Aproximator(nn.Module):
    def __init__(self, input_size: int, output_size: int) -> None:
        super(Aproximator, self).__init__()
        self.linear = nn.Linear(in_features=input_size, out_features=output_size, bias=True)
        

    def forward(self, x: Tensor) -> Tensor:
        return self.linear(x)

In [12]:
# Create an aproximator
aproximator = Aproximator(input_size=4, output_size=2)
# Optimizer
optimizer = torch.optim.SGD(aproximator.parameters(), lr=0.01)